In [1]:
import pandas as pd
import numpy as np
import torch


#loading the data
pd.read_csv('data/train.csv').head()

from torchtext.data import Field


In [2]:
import sys
import csv
maxInt = sys.maxsize
decrement =True
while decrement:
    decrement =False
    try:
        csv.field_size_limit(maxInt)
        sys.maxsize = maxInt
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement =True


In [3]:
tokenize = lambda x: x.split()
#lowercased, whitespace-tokenized, and preprocessed
TEXT = Field(sequential=True, tokenize=tokenize, lower=True)
#labels is numeric already
LABEL = Field(sequential=False, use_vocab=False)

# csv.field_size_limit(sys.maxsize)
# csv.field_size_limit(500 * 1024 * 1024)

from torchtext.data import TabularDataset
trvld_datafields = [("id", None),
                ("comment_text", TEXT), ("toxic", LABEL),
                ("severe_toxic", LABEL), ("threat", LABEL),
                ("obscene", LABEL), ("insult", LABEL),
                ("identity_hate", LABEL)]

trn, vld = TabularDataset.splits(
            path="data",
            train="train.csv", validation="valid.csv",
            format="csv", skip_header=True,
            fields=trvld_datafields)

tst_datafields = [("id", None), ("comment_text", TEXT)]
tst = TabularDataset(
        path="data/test.csv", format="csv", skip_header=True, fields=tst_datafields)

print ('size of the train, dev, test dataset:', len(trn), len(vld), len(tst))
#print (trn.fields.items())
#print (trn[0].comment_text)
#print (tst[0].comment_text) 


size of the train, dev, test dataset: 25 25 33


In [4]:
from torchtext import vocab
vec = vocab.Vectors('E:\\语料\word2vec\\glove.6B\\glove.6B.100d.txt', '.\\data\\')
TEXT.build_vocab(trn, vld, max_size=200, vectors=vec)
print ('size of the vocab and embedding dim:', TEXT.vocab.vectors.shape) #size=202,50 (max vocab size=200+2 for <unk> and <pad>, and glove vector dim=50)
print ('index of the in the vocab:', TEXT.vocab.stoi['the']) #output:2, so the index 2 in vocab is for 'the'
print ('index 0 in the vocab:', TEXT.vocab.itos[0]) #output:<unk>, so the index 0 in vocab is for '<unk>'
#print ('embedding vector for 'the':', TEXT.vocab.vectors[TEXT.vocab.stoi['the']])

#print (TEXT.vocab.freqs.most_common(10))
#print (trn[0].__dict__.keys())
#print (trn[0].comment_text[:5])

#BucketIterator groups sequence of similar lengths text in a batch together to minimize padding! how cool is that!
from torchtext.data import Iterator, BucketIterator
train_iter, val_iter = BucketIterator.splits((trn, vld),
                                            batch_sizes=(3,3),
                                            device='cuda',
                                            sort_key=lambda x: len(x.comment_text), #tell the bucketIterator how to group the sequences
                                            sort_within_batch=False,
                                            repeat=False) #we want to wrap this Iterator layer
test_iter = Iterator(tst, batch_size=3, device='cuda', sort=False, sort_within_batch=False, repeat=False)

print ('number of batch (size: 3):', len(train_iter), len(val_iter)) #output: 9,9. because our train and val data only has 25 examples. since the batch size is 3, it means we have 25/3: 9 batches for each train and val

batch = next(iter(train_iter))
print ('details of batch:', batch)
print ("the content of 'toxic' for the first 3 examples in batch 1:", batch.toxic)
#print ("the content of 'comment_text' for the first 3 examples in batch 1, size depends on the longest sequence:", batch.comment_text)
#print (batch.dataset.fields)


100%|██████████████████████████████████████████████████████████████████████▉| 399977/400001 [00:24<00:00, 16225.53it/s]

size of the vocab and embedding dim: torch.Size([202, 100])
index of the in the vocab: 2
index 0 in the vocab: <unk>
number of batch (size: 3): 9 9
details of batch: 
[torchtext.data.batch.Batch of size 1]
	[.toxic]:[torch.cuda.LongTensor of size 1 (GPU 0)]
	[.severe_toxic]:[torch.cuda.LongTensor of size 1 (GPU 0)]
	[.insult]:[torch.cuda.LongTensor of size 1 (GPU 0)]
	[.threat]:[torch.cuda.LongTensor of size 1 (GPU 0)]
	[.identity_hate]:[torch.cuda.LongTensor of size 1 (GPU 0)]
	[.obscene]:[torch.cuda.LongTensor of size 1 (GPU 0)]
	[.comment_text]:[torch.cuda.LongTensor of size 8x1 (GPU 0)]
the content of 'toxic' for the first 3 examples in batch 1: tensor([1], device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████▉| 399977/400001 [00:40<00:00, 9999.11it/s]

In [5]:
class BatchGenerator:
    def __init__(self, dl, x, y):
        self.dl, self.x, self.y = dl, x, y
    def __len__(self):
        return len(self.dl)
    def __iter__(self):
        for batch in self.dl:
            X = getattr(batch, self.x) #assuming one input
            if self.y is not None: #concat the y into single tensor
                y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y], dim=1).float()
            else:
                y = torch.zeros((1))
            yield (X,y)

train_batch_it = BatchGenerator(train_iter, 'comment_text', ['toxic', 'threat'])
#print ('get data x and y out of batch object:', next(iter(train_batch_it)))
valid_batch_it = BatchGenerator(val_iter, 'comment_text', ['toxic', 'threat'])
test_batch_it = BatchGenerator(test_iter, 'comment_text', None)
#print (next(test_batch_it.__iter__()))


In [8]:
import torch.optim as optim
import torch.nn as nn
from model import BiLSTM

vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 50
out_dim = 2 #only use 'toxic' and 'threat'
pretrained_vec = trn.fields['comment_text'].vocab.vectors
model = BiLSTM(vocab_size, hidden_dim, emb_dim, out_dim, pretrained_vec)
print (model)
model.cuda()

import tqdm
opt = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.BCEWithLogitsLoss()
epochs = 100
train_loss = []
valid_loss = []

for epoch in range(1, epochs+1):
    training_loss = 0.0
    training_corrects = 0
    model.train()
    for x, y in tqdm.tqdm(train_batch_it):
        opt.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)

        loss.backward()
        opt.step()
        training_loss += loss.item() * x.size(0)
    epoch_loss = training_loss/ len(trn)

    val_loss = 0.0
    model.eval()
    for x,y in valid_batch_it:
        preds = model(x)
        loss = criterion(preds, y)
        val_loss += loss.item() * x.size(0)
    val_loss /= len(vld)
    train_loss.append(epoch_loss)
    valid_loss.append(val_loss)
    print ('Epoch: {}, Training loss: {:.4f}, Validation loss: {:.4f}'.format(epoch, epoch_loss, val_loss))
    
#predictions. note that the preds is the probability of the comment belong in each category output
test_preds = []
for x, y in tqdm.tqdm(test_batch_it):
    preds = model(x)
    preds = preds.data.cpu().numpy()
    preds = 1/(1+np.exp(-preds)) #actual output of the model are logits, so we need to pass into sigmoid function
    test_preds.append(preds)
    print (y, ' >>> ',  preds)

test_preds = np.hstack(test_preds)


BiLSTM(
  (embedding): Embedding(202, 100)
  (encoder): LSTM(100, 50, num_layers=2, dropout=0.1, bidirectional=True)
  (linear): Linear(in_features=100, out_features=50, bias=True)
  (predictor): Linear(in_features=50, out_features=2, bias=True)
)



100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 15.92it/s]

Epoch: 1, Training loss: 21.6129, Validation loss: 8.9205



100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 29.68it/s]

Epoch: 2, Training loss: 9.7737, Validation loss: 7.7052


...

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]

tensor([0.])  >>>  [[0.02849705 0.00210586]
 [0.02849705 0.00210586]
 [0.04147828 0.00283769]]
tensor([0.])  >>>  [[0.0284971  0.00210586]
 [0.02853455 0.00211182]
 [0.04454725 0.00305275]]
tensor([0.])  >>>  [[2.8497057e-02 2.1058565e-03]
 [2.8497051e-02 2.1058565e-03]
 [6.7120459e-04 4.7634949e-06]]
tensor([0.])  >>>  [[0.02849706 0.00210586]
 [0.05186025 0.00386484]
 [0.02847693 0.00210329]]
tensor([0.])  >>>  [[0.0285045  0.00210714]
 [0.02849711 0.00210587]
 [0.15458082 0.03585403]]
tensor([0.])  >>>  [[0.05041096 0.00355884]
 [0.02849706 0.00210586]
 [0.38265657 0.32247508]]
tensor([0.])  >>>  [[0.02843609 0.00209789]
 [0.03147018 0.00173809]
 [0.00936708 0.00032664]]
tensor([0.])  >>>  [[0.05988883 0.00520493]
 [0.02849705 0.00210586]
 [0.0283615  0.00208746]]
tensor([0.])  >>>  [[0.02849721 0.00210588]
 [0.07702307 0.00865315]
 [0.02849732 0.0021059 ]]
tensor([0.])  >>>  [[0.02849999 0.00210633]
 [0.02849721 0.00210588]
 [0.03946575 0.00257403]]


 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [00:00<00:00, 97.35it/s]

tensor([0.])  >>>  [[3.8078952e-01 3.2009682e-01]
 [2.8497051e-02 2.1058565e-03]
 [7.2078308e-04 4.6292698e-06]]


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 96.75it/s]


In [11]:
import matplotlib.pyplot as plt
ep = range(1, epochs+1)
print (ep)
plt.plot(ep, train_loss, 'bo', label='Training loss')
plt.plot(ep, valid_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('plt.png')
#print (test_preds)

# import sys
# sys.exit()

# #write predictions to csv
# df = pd.read_csv("data/test.csv") 
# for i, col in enumerate(["toxic", "threat"]):
#     df[col] = test_preds[:, i]
# df.head()


range(1, 101)


In [ ]:
# https://github.com/littleflow3r/bilstm-pytorch-torchtext